In [1]:
import sys
from pathlib import Path
import os

# Get the current working directory (where notebook is executed from)
current_dir = Path.cwd()

# Start from current directory and search upward for project root
# Project root should contain pyproject.toml (not in src/)
project_root = current_dir
max_levels = 10  # Safety limit

for _ in range(max_levels):
    
    # Check if this directory contains pyproject.toml
    if (project_root / "pyproject.toml").exists():
        # Verify it's the actual project root (not a subdirectory)
        # Project root should have pyproject.toml and src/ directory
        if (project_root / "src").exists() and (project_root / "pyproject.toml").exists():
            break
    if project_root == project_root.parent:
        # Reached filesystem root
        break
    project_root = project_root.parent
else:
    # Fallback: go up 3 levels from current directory if we're in src/adapters/ai_chat/
    if "src" in str(current_dir) and "adapters" in str(current_dir):
        project_root = current_dir.parent.parent.parent

# Add project root to Python path (must be absolute path)
project_root = project_root.resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Change working directory to project root
os.chdir(project_root)

print(f"Current directory (before): {current_dir}")
print(f"Project root: {project_root}")
print(f"Working directory (after): {os.getcwd()}")
print(f"pyproject.toml exists: {(project_root / 'pyproject.toml').exists()}")
print(f"src/ exists: {(project_root / 'src').exists()}")


Current directory (before): f:\Dev\interview-service\interview-service\src\adapters\ai_chat
Project root: F:\Dev\interview-service\interview-service
Working directory (after): F:\Dev\interview-service\interview-service
pyproject.toml exists: True
src/ exists: True


In [2]:
from openai import OpenAI
from src.adapters.ai_chat.ai_chat import AIChat
from src.domain.vacancy.vacancy import VacancyInfo
from src.domain.message.message import Message, RoleEnum, TypeEnum
from src.domain.task.task import Task, TaskType
from src.domain.metrics.metrics import MetricsBlock1, MetricsBlock2, MetricsBlock3
import asyncio
import dotenv   
import os
from config import MODEL_NAME, TOKEN_LIMIT

dotenv.load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
# Вариант с доменом без порта (HTTPS):
BASE_URL = "https://llm.t1v.scibox.tech/v1"
# Альтернатива с IP:порт
# BASE_URL = "http://45.145.191.148:4000/v1"

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
print(client)

In [3]:
print(MODEL_NAME, TOKEN_LIMIT)

qwen3-32b-awq 25000


In [4]:
test_interview_plan = """INTERNAL INTERVIEW PLAN ONLY - DO NOT SHARE WITH CANDIDATES  

1. **Warm-up Question (5-7 min)**  
   - [theory] *Explain the primary use cases for Pandas DataFrames vs. NumPy ndarrays. When would you choose one over the other?*  

2. **Theoretical Questions (10-12 min)**  
   - [theory] *What is the purpose of SQLAlchemy's ORM layer? How does it simplify database interactions compared to raw SQL?*  
   - [theory] *Compare TensorFlow and PyTorch. In what scenarios is each framework typically preferred?*  

3. **Core Coding Tasks (30-35 min)**  
   - **Task 1** [coding] *Write Pandas code to load a CSV file, filter rows where column 'A' > 10, and calculate the mean of column 'B'.*  
   - **Task 2** [coding] *Create a NumPy array of shape (3,3) filled with random values. Compute eigenvalues and perform matrix inversion.*  
   - **Task 3** [coding] *Build a simple neural network (1 hidden layer) using PyTorch/TensorFlow to classify the Iris dataset (skeleton code provided). Compile and explain the model.*  

4. **Follow-up & Debugging (10-12 min)**  
   - [theory] *Explain how you would optimize the Pandas code for large datasets.*  
   - [coding] *Debug a provided SQLAlchemy ORM query that fails to join two tables correctly.*  

5. **Wrap-up (3-5 min)**  
   - [theory] *What are the key challenges when integrating NumPy/TensorFlow for GPU-accelerated computations?*  

---  
**Timing Notes**: Adjust based on candidate performance. Prioritize depth in core libraries (Pandas, NumPy) over framework specifics."""

ai_message_content_1 = """Здравствуйте! Добро пожаловать на техническое интервью.  
В ходе интервью вы будете решать задачи на Python (Pandas, Numpy, PyTorch и др.) в встроенной среде. Время ограничено, поэтому работайте аккуратно и оперативно.  
Вы можете задавать уточняющие вопросы по условию задач, но не ожидайте, что я напишу решение за вас — я помогу направить вас в правильное русло. Мы также обсудим ваш подход и код.  
**Важно:** не используйте внешние инструменты (LLM, поисковики), не копируйте код извне — всё должно быть введено вручную. Попытки обойти правила приведут к дезавалидации интервью.  
Чат будет проверен после завершения. Начнём с первой задачи — готовы?
"""

In [5]:
import asyncio
from src.domain.vacancy.vacancy import VacancyInfo
from src.domain.message.message import Message, RoleEnum, TypeEnum
from src.domain.task.task import Task, TaskType
from datetime import timedelta

# Create example vacancy info
vacancy_info = VacancyInfo(
    profession="Python разработчик / Data Scientist",
    position="Junior Python Developer",
    requirements="Pandas, Numpy, Tensorflow, PyTorch, SQLAlchemy",
    questions="",
    tasks=None,
    task_ides=None,
    interview_plan=test_interview_plan,  # Will be generated by create_chat
    duration=timedelta(minutes=30)
)

# Create example chat history
chat_history = [
    # Message(
    #     role=RoleEnum.AI,
    #     type=TypeEnum.RESPONSE,
    #     content=ai_message_content_1
    # ),
    # Message(
    #     role=RoleEnum.AI,
    #     type=TypeEnum.QUESTION,
    #     content="Поясните основные случаи использования Pandas DataFrames и NumPy ndarrays. В каких ситуациях вы выберете один инструмент вместо другого?"
    # ),
    # Message(
    #     role=RoleEnum.USER,
    #     type=TypeEnum.OTHER,
    #     content="А... можете пояснить вопрос? Напомните, что такое ndarray?"
    # )
]


ai_chat = AIChat()



In [6]:
from datetime import timedelta
from src.domain.vacancy.vacancy import VacancyInfo
from src.domain.message.message import Message, RoleEnum, TypeEnum

# --- INTERNAL PLAN (no libraries, ~30 minutes) ---

test_interview_plan = """INTERNAL INTERVIEW PLAN ONLY - DO NOT SHARE WITH CANDIDATES  

1. **Разогрев (3–5 мин)**  
   - [theory] *Что такое список в Python и чем он отличается от кортежа? Приведите пару примеров.*  

2. **Базовая теория (5–7 мин)**  
   - [theory] *Чем список отличается от строки с точки зрения изменяемости и типичных операций?*  
   - [theory] *Что такое функция в Python и зачем нужны аргументы по умолчанию?*  

3. **Основные задачки по коду (15–18 мин)**  
   - **Task 1** [coding] *Написать функцию `sum_to_n(n)`, которая для целого n возвращает сумму чисел от 1 до n. Без сторонних библиотек.*  
   - **Task 2** [coding] *Реализовать классический FizzBuzz: вывести числа от 1 до 100, заменяя кратные 3 на "Fizz", кратные 5 на "Buzz", кратные и 3 и 5 на "FizzBuzz". Только стандартный Python.*  

4. **Дебаг и доработка (3–4 мин)**  
   - [coding] *Попросить кандидата учесть граничные случаи в sum_to_n (n <= 0, некорректный ввод) и обсудить их решение.*  

5. **Финал (2–3 мин)**  
   - [theory] *Короткий вопрос про читаемость кода, выбор имён переменных и простые практики оформления.*  

---  
**Timing Notes**: План рассчитан примерно на 30 минут. При необходимости сокращаем теорию и не расширяем FizzBuzz за пределы базового варианта.
"""

# --- WELCOME MESSAGE (AI) ---

ai_message_content_1 = """Здравствуйте! Добро пожаловать на техническое интервью на позицию Junior Python Developer.  
В этом интервью мы будем работать только с чистым Python — без сторонних библиотек. Вам предстоят несколько коротких теоретических вопросов и 1–2 небольшие задачки по коду.  
Вы можете задавать уточняющие вопросы по условию, но я не буду писать решения за вас — моя задача оценить ваши навыки и помочь наводящими подсказками.  
Пожалуйста, не используйте внешние инструменты (LLM, поисковики) и не копируйте заранее подготовленный код. Всё решение должно быть введено вами вручную.  
Если всё понятно, давайте начнём с небольшого разогрева.
"""

# --- VACANCY INFO MOCK ---

vacancy_info = VacancyInfo(
    profession="Python разработчик",
    position="Junior Python Developer",
    requirements="Базовый Python, типы данных, условия, циклы, функции. Без внешних библиотек.",
    questions="Базовая теория по спискам, строкам, функциям, простые задачки на циклы и условия.",
    tasks=None,
    task_ides=None,
    interview_plan=test_interview_plan,
    duration=timedelta(minutes=30),
)

# --- CHAT HISTORY MOCK (FULL 30-MIN INTERVIEW) ---

chat_history = [
    # 1. Приветствие и правила
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.RESPONSE,
        content=ai_message_content_1,
    ),

    # 2. Разогрев – теоретический вопрос
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.QUESTION,
        content="Для начала небольшой вопрос: что такое список в Python и чем он отличается от кортежа? Приведите пару простых примеров.",
    ),
    Message(
        role=RoleEnum.USER,
        type=TypeEnum.ANSWER,
        content=(
            "Список — это такой контейнер, где можно хранить разные значения. "
            "Кортеж, кажется, примерно то же самое, только в круглых скобках. "
            "Честно говоря, не помню точно, в чём ещё разница, кроме скобок."
        ),
    ),
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.HINT,
        content=(
            "В целом верно, что и список, и кортеж хранят коллекцию значений. "
            "Ключевая разница — изменяемость: список можно изменять (добавлять, удалять, менять элементы), "
            "а кортеж — нет. Это важно, например, при передаче данных, которые не должны меняться."
        ),
    ),

    # 3. Теория – ещё один вопрос, кандидат слабоват
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.QUESTION,
        content="А чем список отличается от строки с точки зрения операций и изменяемости?",
    ),
    Message(
        role=RoleEnum.USER,
        type=TypeEnum.ANSWER,
        content=(
            "Строка — это тоже как массив символов. Кажется, её тоже можно менять по индексу, "
            "но я так почти не делал. В основном я к строкам просто что-то дописывал через '+'."
        ),
    ),
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.CHECK_SOLUTION,
        content=(
            "Тут есть неточность: строки в Python **неизменяемы**, как и кортежи. "
            "Когда вы 'дописываете' к строке, на самом деле создаётся новая строка. "
            "Список же можно менять по индексу и модифицировать на месте. Это важно для понимания работы памяти и скорости."
        ),
    ),

    # 4. Переход к коду – Task 1
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.TASK,
        content=(
            "[coding][python]\n"
            "Задание 1. Напишите функцию `sum_to_n(n)`, которая возвращает сумму чисел от 1 до n включительно. "
            "Считайте, что n — целое число. Сторонние библиотеки использовать нельзя."
        ),
    ),
    Message(
        role=RoleEnum.USER,
        type=TypeEnum.SOLUTION,
        content=(
            "Я сделал так: \n"
            "def sum_to_n(n):\n"
            "    s = 0\n"
            "    for i in range(1, n+1):\n"
            "        s += i\n"
            "    return s\n"
            "Кажется, должно работать."
        ),
    ),
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.CHECK_SOLUTION,
        content=(
            "Подход корректный для положительных n. Функция вернёт правильную сумму. "
            "Попробуйте подумать, что будет, если n = 0 или n < 0. "
            "Как бы вы обработали такие случаи?"
        ),
    ),
    Message(
        role=RoleEnum.USER,
        type=TypeEnum.ANSWER,
        content=(
            "Можно в начале функции проверить: если n <= 0, то вернуть 0 или, например, "
            "поднять исключение. Я бы, наверное, вернул 0, чтобы не падать."
        ),
    ),
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.RESPONSE,
        content=(
            "Да, проверка в начале — хороший вариант. Для интервью достаточно вернуть 0 и явно это задокументировать. "
            "По основному случаю решение верное."
        ),
    ),

    # 5. Task 2 – FizzBuzz
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.TASK,
        content=(
            "[coding][python]\n"
            "Задание 2. Реализуйте FizzBuzz: выведите числа от 1 до 100, "
            "заменяя числа, кратные 3, на 'Fizz', кратные 5 — на 'Buzz', "
            "а кратные и 3, и 5 — на 'FizzBuzz'. Используйте только базовый Python."
        ),
    ),
    Message(
        role=RoleEnum.USER,
        type=TypeEnum.SOLUTION,
        content=(
            "Сделал примерно так:\n"
            "for i in range(1, 101):\n"
            "    if i % 3 == 0 and i % 5 == 0:\n"
            "        print('FizzBuzz')\n"
            "    elif i % 3 == 0:\n"
            "        print('Fizz')\n"
            "    elif i % 5 == 0:\n"
            "        print('Buzz')\n"
            "    else:\n"
            "        print(i)\n"
        ),
    ),
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.CHECK_SOLUTION,
        content=(
            "Это классическая и вполне корректная реализация. Логика условий верная, порядок проверок тоже. "
            "Можно было бы чуть упростить, но для уровня junior такое решение выглядит хорошо."
        ),
    ),

    # 6. Небольшой follow-up по коду / читаемости
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.QUESTION,
        content=(
            "Как бы вы улучшили читаемость кода в этих задачах? "
            "Можете привести пример, какие имена переменных или функции вы бы выбрали."
        ),
    ),
    Message(
        role=RoleEnum.USER,
        type=TypeEnum.ANSWER,
        content=(
            "Наверное, я бы в FizzBuzz вынес логику в отдельную функцию, например fizzbuzz(n), "
            "а уже потом вызывал её в цикле. Переменную i можно переименовать в number, чтобы было понятнее. "
            "В сумме до n можно добавить докстринг к функции."
        ),
    ),
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.RESPONSE,
        content=(
            "Отличная идея с вынесением логики в функцию и более говорящими именами. "
            "Это как раз то, что ожидается от junior-разработчика: рабочий код плюс базовое внимание к читаемости."
        ),
    ),

    # 7. Финальное резюме от интервьюера
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.RESPONSE,
        content=(
            "На этом мы закончим. В целом вы неплохо справились с практическими задачами: "
            "код корректный, без лишней сложности. В теории по типам данных есть пробелы "
            "(особенно по изменяемости строк и кортежей), но это можно подтянуть. "
            "Спасибо за интервью!"
        ),
    ),
]


In [7]:
import asyncio
from datetime import timedelta

from src.adapters.ai_chat.ai_chat import AIChat
from src.domain.vacancy.vacancy import VacancyInfo
from src.domain.message.message import Message, RoleEnum, TypeEnum
from src.domain.task.task import Task, TaskType, TaskLanguage
from src.domain.test.test import CodeTestSuite, CodeTestCase  # type hints only

# ---------- Mock interview plan ----------

test_interview_plan = """INTERNAL INTERVIEW PLAN ONLY - DO NOT SHARE WITH CANDIDATES

1. Разогрев (5 минут)
   - [theory] Кратко объяснить, чем список (list) отличается от кортежа (tuple) в Python 
     и когда что лучше использовать.

2. Основная задача (20 минут)
   - [coding] Модифицированная задача two-sum без сторонних библиотек:
     Дано: список целых чисел nums и целевое число target.
     Требуется: найти ДВА индекса i и j (i < j) такие, что nums[i] + nums[j] == target.
     Если подходящей пары нет — вернуть -1 -1.

3. Wrap-up (5 минут)
   - [theory] Попросить кандидата кратко объяснить алгоритм и оценку сложности.
"""

# ---------- Vacancy info ----------

vacancy_info = VacancyInfo(
    profession="Python разработчик",
    position="Junior Python Developer",
    requirements="Уверенное владение Python, базовые алгоритмы и структуры данных, умение писать чистый код.",
    questions="Базовые вопросы по Python, списки, кортежи, циклы, функции.",
    tasks=[
        "Разогрев: теоретический вопрос по базовым структурам данных в Python. [theory]",
        "Основная задача: простая алгоритмическая задача без сторонних библиотек (вариант two-sum). [coding]",
    ],
    task_ides=["алгоритмическая задача по поиску пары чисел по сумме"],
    interview_plan=test_interview_plan,
    duration=timedelta(minutes=30),
)

# ---------- Chat history (кандидат ответил на теорию, сейчас переходим к коду) ----------

ai_welcome_message = (
    "Здравствуйте! Это короткое техническое интервью на позицию Junior Python Developer. "
    "Сначала обсудим базовую теорию, затем перейдём к одной простой алгоритмической задаче без библиотек. "
    "Можете задавать уточняющие вопросы по условию, но я не буду писать решение за вас."
)

ai_theory_question = (
    "Первый вопрос: расскажите, чем список (list) отличается от кортежа (tuple) в Python "
    "и когда вы предпочли бы использовать каждый из них?"
)

user_theory_answer = (
    "Список — изменяемый, в него можно добавлять и удалять элементы, "
    "кортеж — неизменяемый. Списки использую, когда структура данных должна меняться, "
    "а кортежи — когда важно зафиксировать набор значений, например координаты."
)

ai_theory_feedback_and_move_to_code = (
    "Отличный ответ: вы правильно выделили изменяемость и типичные случаи использования. "
    "Теперь перейдём к основной задаче по алгоритмам на Python без сторонних библиотек."
)

ai_code_task_intro = (
    "Задача: у вас есть список целых чисел nums и число target. "
    "Нужно найти ДВА индекса i и j (i < j), такие что nums[i] + nums[j] == target. "
    "Если подходящей пары нет — верните -1 -1. "
    "Реализуйте функцию two_sum_modified(nums: list[int], target: int) и выведите индексы через пробел."
)

chat_history = [
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.RESPONSE,
        content=ai_welcome_message,
    ),
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.QUESTION,
        content=ai_theory_question,
    ),
    Message(
        role=RoleEnum.USER,
        type=TypeEnum.ANSWER,
        content=user_theory_answer,
    ),
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.RESPONSE,
        content=ai_theory_feedback_and_move_to_code,
    ),
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.TASK,
        content=ai_code_task_intro,
    ),
]

# ---------- Current coding task object ----------

task = Task(
    type=TaskType.CODE,
    language=TaskLanguage.PYTHON,
    description=(
        "Реализуйте функцию two_sum_modified(nums: list[int], target: int), "
        "которая находит ДВА индекса i и j (i < j), такие что nums[i] + nums[j] == target. "
        "Если пары нет, выведите \"-1 -1\". Вход и выход реализуются через stdin/stdout."
    ),
)

# ---------- Candidate solution (plausible correct implementation) ----------

candidate_solution_code = """\
def two_sum_modified(nums, target):
    n = len(nums)
    for i in range(n):
        for j in range(i + 1, n):
            if nums[i] + nums[j] == target:
                print(i, j)
                return
    print(-1, -1)


if __name__ == "__main__":
    import sys
    data = sys.stdin.read().strip().split()
    if not data:
        print("-1 -1")
    else:
        *nums_str, target_str = data
        nums = [int(x) for x in nums_str]
        target = int(target_str)
        two_sum_modified(nums, target)
"""

ai_chat = AIChat()

# ---------- Test for create_test_suite ----------

async def test_create_test_suite() -> CodeTestSuite:
    ai_chat_local = AIChat()

    print("=== Running create_test_suite ===\n")

    suite = await ai_chat_local.create_test_suite(
        vacancy_info=vacancy_info,
        chat_history=chat_history,
        task=task,
    )

    print("=== Test Suite Summary ===")
    print(f"Task ID: {suite.task_id}")
    print(f"Total tests: {len(suite.tests)}")

    visible = [t for t in suite.tests if not t.is_hidden]
    hidden = [t for t in suite.tests if t.is_hidden]

    print(f"Visible tests: {len(visible)}")
    print(f"Hidden tests:  {len(hidden)}\n")

    print("--- Visible tests ---")
    for t in visible:
        print(f"[{t.id}] input={repr(t.input_data)} -> expected={repr(t.expected_output)}")

    print("\n--- Hidden tests (ids only) ---")
    for t in hidden:
        print(f"[{t.id}] (hidden)")

    return suite

# ---------- Test for check_solution, reusing the created suite ----------

async def test_check_solution(suite: CodeTestSuite) -> None:
    ai_chat_local = AIChat()

    # Imitate that the external runner already executed tests and filled correctness flags.
    # Here we assume the candidate solution performed well: almost all tests passed.
    if suite.tests:
        # Mark all as passed by default
        for t in suite.tests:
            t.correct = True

        # Optionally, make one hidden test fail to simulate a subtle bug:
        # if len(suite.tests) > 2:
        #     suite.tests[-1].correct = False

    # Extend chat history with a short user message like "please check my solution"
    chat_with_solution_request = chat_history + [
        Message(
            role=RoleEnum.USER,
            type=TypeEnum.SOLUTION,
            content="Я написал решение в редакторе, пожалуйста, проверьте его на тестах.",
        )
    ]

    print("\n=== Running check_solution ===\n")

    stream, ai_msg = await ai_chat_local.check_solution(
        vacancy_info=vacancy_info,
        chat_history=chat_with_solution_request,
        task=task,
        solution=candidate_solution_code,
        tests=suite,
    )

    chunks: list[str] = []
    async for chunk in stream:
        chunks.append(chunk)
        print(chunk, end="", flush=True)

    ai_msg.content = "".join(chunks)

    print("\n\n=== Final check_solution message ===")
    print(f"Role: {ai_msg.role}")
    print(f"Type: {ai_msg.type}")
    print(f"Content:\n{ai_msg.content}")


# ---------- Run tests ----------

print("Testing create_test_suite...")
suite = await test_create_test_suite()

print("\n" + "=" * 50 + "\n")




Testing create_test_suite...
=== Running create_test_suite ===

=== Test Suite Summary ===
Task ID: task_without_id
Total tests: 10
Visible tests: 4
Hidden tests:  6

--- Visible tests ---
[t1] input='2 7\n9\n' -> expected='0 1\n'
[t2] input='3 2 4\n6\n' -> expected='1 2\n'
[t3] input='3 3\n6\n' -> expected='0 1\n'
[t4] input='1 2 3\n7\n' -> expected='-1 -1\n'

--- Hidden tests (ids only) ---
[t5] (hidden)
[t6] (hidden)
[t7] (hidden)
[t8] (hidden)
[t9] (hidden)
[t10] (hidden)




In [8]:
print("Testing check_solution...")
await test_check_solution(suite)

Testing check_solution...

=== Running check_solution ===



Ваше решение прошло 4 из 10 видимых тестов, но не сдало скрытые проверки. Например, для входа `-1 0` и `target=-1` алгоритм возвращает правильный ответ `0 1`, но возможно есть ошибки в обработке дубликатов или сложных комбинаций. Рассмотрите оптимизацию через хэш-таблицу (словарь) для хранения значений и их индексов — это сократит сложность с O(n²) до O(n). Проверьте также, возвращается ли всегда первая подходящая пара, а не любая.

=== Final check_solution message ===
Role: ai
Type: check_solution
Content:


Ваше решение прошло 4 из 10 видимых тестов, но не сдало скрытые проверки. Например, для входа `-1 0` и `target=-1` алгоритм возвращает правильный ответ `0 1`, но возможно есть ошибки в обработке дубликатов или сложных комбинаций. Рассмотрите оптимизацию через хэш-таблицу (словарь) для хранения значений и их индексов — это сократит сложность с O(n²) до O(n). Проверьте также, возвращается ли всегда первая подходящая пара, 

In [9]:
async def test_generate_welcome_message():
    ai_chat = AIChat()
    
    # First create chat to get updated vacancy with interview_plan
    # updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    updated_vacancy = vacancy_info
    print("\n=== Welcome Message (streaming) ===\n")
    welcome_chunks = []
    
    # ❌ before:
    # async for chunk in ai_chat.generate_welcome_message(updated_vacancy, chat_history):

    # ✅ after:
    stream = await ai_chat.generate_welcome_message(updated_vacancy, chat_history)
    async for chunk in stream:
        welcome_chunks.append(chunk)
        print(chunk, end="", flush=True)
    
    print("\n\n=== Full Welcome Message ===")
    full_welcome = "".join(welcome_chunks)
    print(full_welcome)
    
print("Testing generate_welcome_message...")
await test_generate_welcome_message()

Testing generate_welcome_message...

=== Welcome Message (streaming) ===



Добро пожаловать на техническое интервью! В ходе следующих 30 минут вы решите одну-две задачи на Python, а мы обсудим ваши подходы и код. Задачи будут проверять знание базовых структур данных, алгоритмов и практик работы с Python. Время ограничено, поэтому старайтесь быть эффективными. 

Важно: 
- Все решения нужно писать самостоятельно, без использования внешних инструментов (LLM, сторонние библиотеки и т.п.).
- Копирование кода извне или попытки обойти правила приведут к дискаualификации.
- Вы можете задавать уточняющие вопросы по условию, но я не буду писать готовые решения.

Первая задача будет отправлена через несколько секунд. Удачи!

=== Full Welcome Message ===


Добро пожаловать на техническое интервью! В ходе следующих 30 минут вы решите одну-две задачи на Python, а мы обсудим ваши подходы и код. Задачи будут проверять знание базовых структур данных, алгоритмов и практик работы с Python. Время огранич

In [12]:
%pwd

'F:\\Dev\\interview-service\\interview-service'

In [8]:
# from datetime import timedelta
# from src.domain.metrics.metrics import MetricsBlock1
# from src.adapters.ai_chat.ai_chat import AIChat

# Mock MetricsBlock1 for a ~30-minute interview
metrics_block1 = MetricsBlock1(
    time_spent=timedelta(minutes=29),
    time_per_task=timedelta(minutes=7),  # e.g. 4 tasks roughly
    answers_count=10,
    copy_paste_suspicion=1,  # low suspicion
)

# Test create_metrics function
async def test_create_metrics():
    ai_chat = AIChat()

    print("=== Running create_metrics ===\n")

    block1, block2, block3 = await ai_chat.create_metrics(
        vacancy_info=vacancy_info,
        chat_history=chat_history,
        metrics_block1=metrics_block1,
    )

    # ---- Block 1 (input, just to confirm) ----
    print("=== MetricsBlock1 (input) ===")
    print(f"time_spent:        {block1.time_spent}")
    print(f"time_per_task:     {block1.time_per_task}")
    print(f"answers_count:     {block1.answers_count}")
    print(f"copy_paste_suspicion: {block1.copy_paste_suspicion}")
    print()

    # ---- Block 2 (LLM-generated) ----
    print("=== MetricsBlock2 (LLM) ===")
    print(f"summary:\n{block2.summary}\n")
    print(f"clarity_score:      {block2.clarity_score}")
    print(f"completeness_score: {block2.completeness_score}")
    print(f"feedback_response:  {block2.feedback_response}")
    print(f"tech_fit_level:     {block2.tech_fit_level.value}")
    print(f"tech_fit_comment:   {block2.tech_fit_comment}")
    print()

    # ---- Block 3 (LLM-generated) ----
    print("=== MetricsBlock3 (LLM) ===")
    print(f"strengths:\n{block3.strengths}\n")
    print(f"weaknesses:\n{block3.weaknesses}\n")
    print(f"cheating_summary:\n{block3.cheating_summary}\n")
    print(f"seniority_guess: {block3.seniority_guess.value}")
    print(f"recommendation:  {block3.recommendation.value}")
    
    return block1, block2, block3


print("Testing create_metrics...")
block1, block2, block3 = await test_create_metrics()


Testing create_metrics...
=== Running create_metrics ===

=== MetricsBlock1 (input) ===
time_spent:        0:29:00
time_per_task:     0:07:00
answers_count:     10
copy_paste_suspicion: 1

=== MetricsBlock2 (LLM) ===
summary:
Кандидат показал базовое понимание Python, особенно в практической реализации задач (sum_to_n и FizzBuzz). В теоретических вопросах имелись неточности по изменяемости строк и кортежей, но он улучшил ответы после подсказок. В задачах учитывал граничные случаи и предлагал улучшения читаемости кода. Коммуникация была понятной, хотя в начальных вопросах были сомнения.

clarity_score:      3
completeness_score: 4
feedback_response:  Кандидат активно реагировал на подсказки: исправил ошибки в понимании изменяемости строк и добавил обработку граничных случаев в функцию sum_to_n. Это показывает способность принимать обратную связь и адаптироваться.
tech_fit_level:     medium
tech_fit_comment:   Кандидат справился с базовыми задачами на циклы и условия, реализовал корректн

In [7]:
# Chat history where the candidate has already answered
# the warm-up + all theoretical questions.
# The last message is the candidate's answer about TensorFlow vs PyTorch,
# so the next step according to the plan should be Core Coding Task 1.

chat_history = [
    # Greeting / intro
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.RESPONSE,
        content=ai_message_content_1,
    ),

    # Warm-up question: Pandas vs NumPy
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.QUESTION,
        content=(
            "Поясните основные случаи использования Pandas DataFrames и NumPy ndarrays. "
            "В каких ситуациях вы выберете один инструмент вместо другого?"
        ),
    ),
    Message(
        role=RoleEnum.USER,
        type=TypeEnum.ANSWER,
        content=(
            "Pandas DataFrame я бы использовал для работы с табличными данными — когда есть "
            "строки и столбцы, фильтрация, группировки, агрегации, merge/join. "
            "NumPy ndarray — когда нужны векторизованные численные вычисления, линейная алгебра, "
            "работа с многомерными массивами и матрицами. Обычно я загружаю данные в DataFrame, "
            "а при необходимости могу вытащить values в ndarray для матричных операций."
        ),
    ),

    # Короткий фидбек по первому вопросу
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.CHECK_SOLUTION,
        content=(
            "Хорошо, вы верно разделили области применения: Pandas для табличных операций и "
            "анализa данных, NumPy для низкоуровневых численных вычислений и линейной алгебры. "
            "Перейдём к следующим теоретическим вопросам."
        ),
    ),

    # Теоретический вопрос 1: SQLAlchemy ORM
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.QUESTION,
        content=(
            "Расскажите, для чего нужен ORM-слой в SQLAlchemy и чем работа через ORM удобнее, "
            "чем использование сырых SQL-запросов?"
        ),
    ),
    Message(
        role=RoleEnum.USER,
        type=TypeEnum.ANSWER,
        content=(
            "ORM в SQLAlchemy позволяет описывать таблицы как Python-классы и работать с ними "
            "как с объектами. Вместо ручной сборки SQL-запросов я пишу выражения на Python, а "
            "SQLAlchemy генерирует SQL под конкретную СУБД. Это снижает количество шаблонного кода, "
            "даёт типизацию на уровне моделей и упрощает миграции между базами. При этом при "
            "необходимости можно всегда написать raw SQL."
        ),
    ),

    # Короткий фидбек по SQLAlchemy-вопросу
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.CHECK_SOLUTION,
        content=(
            "Да, верно: ORM абстрагирует SQL, позволяет работать с моделями, а не строками запросов, "
            "и упрощает переносимость и сопровождение кода. Теперь последний теоретический вопрос."
        ),
    ),

    # Теоретический вопрос 2: TensorFlow vs PyTorch
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.QUESTION,
        content=(
            "Сравните TensorFlow и PyTorch. В каких сценариях обычно используют каждый из этих "
            "фреймворков и чем они отличаются с точки зрения разработки?"
        ),
    ),

    # Последнее сообщение — ответ кандидата.
    # После этого create_task должен взять Task 1 из секции Core Coding Tasks.
    Message(
        role=RoleEnum.USER,
        type=TypeEnum.ANSWER,
        content=(
            "PyTorch чаще используют для научных исследований и прототипирования, потому что у него "
            "динамический вычислительный граф и более 'питоничный' стиль кода — удобно дебажить и "
            "экспериментировать. TensorFlow больше ориентирован на продакшен и развёртывание: там есть "
            "TensorFlow Serving, TFLite, хорошая интеграция с экосистемой Google. Сейчас границы "
            "размылись, но в целом: PyTorch — быстрее стартовать и iterировать модель, TensorFlow — "
            "удобнее, когда нужно выстроить целую продакшен-пайплайн."
        ),
    ),
]


In [9]:
# Test create_response function
async def test_create_response():
    ai_chat = AIChat()
    
    # Example task for this interview turn
    task = Task(
        type=TaskType.THEORY,
        language="Python",
        description="Поясните основные случаи использования Pandas DataFrames и NumPy ndarrays. В каких ситуациях вы выберете один инструмент вместо другого?"
    )

    stream, user_msg, ai_msg = await ai_chat.create_response(
        vacancy_info=vacancy_info,
        chat_history=chat_history,
        task=task,
    )

    # Fill user message content from the last USER message in history
    last_user_msg = next((m for m in reversed(chat_history) if m.role == RoleEnum.USER), None)
    if last_user_msg:
        user_msg.content = last_user_msg.content

    print("=== Classification ===")
    print(f"User message type:      {user_msg.type}")
    print(f"Assistant message type: {ai_msg.type}")

    print("\n=== AI response (streaming) ===\n")
    chunks: list[str] = []
    async for chunk in stream:
        chunks.append(chunk)
        print(chunk, end="", flush=True)

    ai_msg.content = "".join(chunks)

    print("\n\n=== Final AI message ===")
    print(f"Role:    {ai_msg.role}")
    print(f"Type:    {ai_msg.type}")
    print(f"Content: {ai_msg.content}")


print("Testing create_response...")
await test_create_response()


Testing create_response...
=== Classification ===
User message type:      question
Assistant message type: hint

=== AI response (streaming) ===

  
NumPy ndarray (N-мерный массив) — это основной тип данных в NumPy для работы с числовыми данными, оптимизированный для высокой производительности. Он представляет собой однородную структуру (все элементы одного типа) и поддерживает векторизованные операции. Pandas DataFrame — это структура, похожая на таблицу в Excel, с поддержкой разнотиповых колонок, индексации и операций, характерных для работы с данными (фильтрация, группировка и т.д.).  

Попробуйте сформулировать, в каких сценариях использование одного из этих инструментов предпочтительнее. Например: когда нужно выполнять математические вычисления над массивами данных — NumPy, а когда работать с табличными данными, содержащими разнородные типы — Pandas.

=== Final AI message ===
Role:    ai
Type:    hint
Content:   
NumPy ndarray (N-мерный массив) — это основной тип данных в NumPy дл

In [9]:
# Test create_task function
async def test_create_task():
    ai_chat = AIChat()

    # Ask AI to generate the next task
    stream, task = await ai_chat.create_task(
        vacancy_info=vacancy_info,
        chat_history=chat_history,
    )

    print("=== Task metadata from <ctrl> ===")
    print(f"Task type:      {task.type}")
    print(f"Task language:  {task.language}")

    print("\n=== Task description (streaming) ===\n")
    chunks: list[str] = []
    async for chunk in stream:
        chunks.append(chunk)
        print(chunk, end="", flush=True)

    # Store full description back into the Task object
    task.description = "".join(chunks)

    print("\n\n=== Final Task object ===")
    print(f"Task type:      {task.type}")
    print(f"Task language:  {task.language}")
    print(f"Description:\n{task.description}")

    # So you can use it later in the notebook/code
    return task


print("Testing create_task...")
generated_task = await test_create_task()


Testing create_task...
=== Task metadata from <ctrl> ===
Task type:      TaskType.CODE
Task language:  TaskLanguage.PYTHON

=== Task description (streaming) ===


Напишите код на Pandas для загрузки CSV-файла из стандартного ввода, фильтрации строк, где значение в колонке 'A' превышает 10, и вычисления среднего значения колонки 'B' среди оставшихся строк. Результат выведите в стандартный вывод. Учтите, что файл содержит заголовок с именами колонок.

=== Final Task object ===
Task type:      TaskType.CODE
Task language:  TaskLanguage.PYTHON
Description:

Напишите код на Pandas для загрузки CSV-файла из стандартного ввода, фильтрации строк, где значение в колонке 'A' превышает 10, и вычисления среднего значения колонки 'B' среди оставшихся строк. Результат выведите в стандартный вывод. Учтите, что файл содержит заголовок с именами колонок.


In [11]:
stream

NameError: name 'stream' is not defined

In [8]:

# Test create_chat function
async def test_create_chat():
    ai_chat = AIChat()
    
    # create_chat returns updated VacancyInfo with interview_plan
    updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    
    print("=== Updated Vacancy Info ===")
    print(f"Profession: {updated_vacancy.profession}")
    print(f"Position: {updated_vacancy.position}")
    print(f"\nInterview Plan (first 300 chars):")
    print(updated_vacancy.interview_plan + "..." if len(updated_vacancy.interview_plan) > 300 else updated_vacancy.interview_plan)
    print(f"\nFull Interview Plan Length: {len(updated_vacancy.interview_plan)} characters")

# Test generate_welcome_message function
async def test_generate_welcome_message():
    ai_chat = AIChat()
    
    # First create chat to get updated vacancy with interview_plan
    updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    
    print("\n=== Welcome Message (streaming) ===\n")
    welcome_chunks = []
    
    # ❌ before:
    # async for chunk in ai_chat.generate_welcome_message(updated_vacancy, chat_history):

    # ✅ after:
    stream = await ai_chat.generate_welcome_message(updated_vacancy, chat_history)
    async for chunk in stream:
        welcome_chunks.append(chunk)
        print(chunk, end="", flush=True)
    
    print("\n\n=== Full Welcome Message ===")
    full_welcome = "".join(welcome_chunks)
    print(full_welcome)

# Run the async functions
print("Testing create_chat...")
await test_create_chat()

print("\n" + "="*50 + "\n")

print("Testing generate_welcome_message...")
await test_generate_welcome_message()


Testing create_chat...
=== Updated Vacancy Info ===
Profession: Python разработчик / Data Scientist
Position: Junior Python Developer

Interview Plan (first 300 chars):
INTERNAL INTERVIEW PLAN ONLY (DO NOT SHARE WITH CANDIDATE)

1. **Warm-up (5-10 min)**  
   - [theory] Explain how to handle missing data in Pandas DataFrames. Demonstrate .dropna() vs .fillna() with example scenarios.  

2. **Core Libraries Assessment (30-40 min)**  
   - [coding] Numpy task: Write a function to normalize a 2D array (row-wise) and handle division by zero.  
   - [theory] Compare Tensorflow and PyTorch: When would you choose one over the other? Discuss dynamic vs static computation graphs.  

3. **ML Frameworks Practical (20-30 min)**  
   - [coding] PyTorch task: Implement a single-layer neural network for binary classification (define model, loss function, and optimizer).  

4. **Database Integration (15-20 min)**  
   - [theory] Explain SQLAlchemy's ORM approach. How does it differ from raw SQL? Provi

In [ ]:
# Test create_response function (for reference)
async def test_create_response():
    ai_chat = AIChat()
    
    # First create chat to get updated vacancy
    updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    
    # Then create response with task
    print("\n=== Response to Task (streaming) ===\n")
    response_chunks = []
    stream = await ai_chat.create_response(updated_vacancy, chat_history, task)
    async for chunk in stream:
        response_chunks.append(chunk)
        print(chunk, end="", flush=True)
    
    print("\n\n=== Full Response ===")
    full_response = "".join(response_chunks)
    print(full_response)

# Run the async function
await test_create_response()


In [6]:
# Test stream_task function
async def test_stream_task():
    ai_chat = AIChat()
    
    # First create chat to get updated vacancy with interview_plan
    # updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    vacancy_info.interview_plan = test_interview_plan
    updated_vacancy = vacancy_info
    print(updated_vacancy.interview_plan)
    # Then stream the task
    print("\n=== Streaming Task Description ===\n")
    task_chunks = []
    
    stream = await ai_chat.stream_task(updated_vacancy, chat_history)
    async for chunk in stream:
        task_chunks.append(chunk)
        print(chunk, end="", flush=True)
    
    print("\n\n=== Full Task Description ===")
    full_task = "".join(task_chunks)
    print(full_task)

# Run the async function
await test_stream_task()


INTERNAL INTERVIEW PLAN ONLY - DO NOT SHARE WITH CANDIDATES  

1. **Warm-up Question (5-7 min)**  
   - [theory] *Explain the primary use cases for Pandas DataFrames vs. NumPy ndarrays. When would you choose one over the other?*  

2. **Theoretical Questions (10-12 min)**  
   - [theory] *What is the purpose of SQLAlchemy's ORM layer? How does it simplify database interactions compared to raw SQL?*  
   - [theory] *Compare TensorFlow and PyTorch. In what scenarios is each framework typically preferred?*  

3. **Core Coding Tasks (30-35 min)**  
   - **Task 1** [coding] *Write Pandas code to load a CSV file, filter rows where column 'A' > 10, and calculate the mean of column 'B'.*  
   - **Task 2** [coding] *Create a NumPy array of shape (3,3) filled with random values. Compute eigenvalues and perform matrix inversion.*  
   - **Task 3** [coding] *Build a simple neural network (1 hidden layer) using PyTorch/TensorFlow to classify the Iris dataset (skeleton code provided). Compile and ex